In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/zip'

In [4]:
! pip install pygeohash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6167 sha256=a166df6eaac5135872d88d504c5e7e4882a13c6d9522699b80f1b6ff1544871f
  Stored in directory: /root/.cache/pip/wheels/e2/ed/86/be7141d743cfb4aebac76406187154cd78fe9e6535d872187a
Successfully built pygeohash


In [5]:
import numpy as np
import pandas as pd
from pandas import*
import datetime
import glob

# Data Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data pre-processing libraries
from sklearn.preprocessing import LabelEncoder
import pygeohash as gh

In [6]:
import os, zipfile

dir_name = '/content/drive/MyDrive/zip'

extension = ".zip"



os.chdir('/content/drive/MyDrive/zip') # change directory from working dir to dir with files

for item in os.listdir(dir_name): 
  if item.endswith(extension): 
    file_name = os.path.abspath(item) # get full path of files
    zip_ref = zipfile.ZipFile(file_name) # create zipfile object
    zip_ref.extractall('/content/drive/MyDrive/zip/csv1') # extract file to dir

zip_ref.close() # close file

#os.remove(file_name) # de

In [7]:
path =  '/content/drive/MyDrive/zip'

In [8]:
file_list = glob.glob(path + '/csv1/*.*')
print(file_list)

['/content/drive/MyDrive/zip/csv1/AIS_2022_01_01.csv', '/content/drive/MyDrive/zip/csv1/AIS_2022_01_02.csv', '/content/drive/MyDrive/zip/csv1/AIS_2022_01_03.csv', '/content/drive/MyDrive/zip/csv1/AIS_2022_01_04.csv', '/content/drive/MyDrive/zip/csv1/AIS_2022_01_06.csv', '/content/drive/MyDrive/zip/csv1/AIS_2022_01_05.csv']


In [9]:
my_list = []
new_path = path + '/csv1/*.csv'
for file in glob.glob(new_path):
  print(file)
  a = pd.read_csv(file, encoding="ISO-8859-1")
  my_list.append(a)

/content/drive/MyDrive/zip/csv1/AIS_2022_01_01.csv
/content/drive/MyDrive/zip/csv1/AIS_2022_01_02.csv
/content/drive/MyDrive/zip/csv1/AIS_2022_01_03.csv
/content/drive/MyDrive/zip/csv1/AIS_2022_01_04.csv
/content/drive/MyDrive/zip/csv1/AIS_2022_01_06.csv
/content/drive/MyDrive/zip/csv1/AIS_2022_01_05.csv


In [13]:
def drop_missing_values(data):
  data.isna().mean().round(4)*100
  data.dropna(inplace=True)
  data.isna().sum()
  data.drop_duplicates()
  

In [16]:
def drop_unwanted_columns(data):
  data.drop(columns=['Heading',
       'VesselName', 'IMO', 'CallSign', 'Status', 'Length',
       'Width', 'Draft', 'Cargo', 'TransceiverClass'], axis = 1, inplace=True)

In [12]:
def drop_invalid_mmsi(data):
  
  invalid_entries=[111111111,222222222,333333333,444444444,555555555,666666666,777777777,888888888,999999999, 123456789, 987654321]
  invalid = data[data.MMSI.isin(invalid_entries)]
  data.drop(invalid.index, axis=0, inplace=True)
  #print(data)

In [17]:
def drop_invalid_mmsi(data):
  
  invalid_entries=[111111111,222222222,333333333,444444444,555555555,666666666,777777777,888888888,999999999, 123456789, 987654321]
  invalid = data[data.MMSI.isin(invalid_entries)]
  data.drop(invalid.index, axis=0, inplace=True)
  #print(data)

In [18]:
def invalid_mmsi_length(data):
  data['MMSI_length']= data['MMSI'].map(str).apply(len)
  data = data[data['MMSI_length']==9]
  data.drop('MMSI_length', axis=1, inplace=True)

In [19]:
def zero_sog(data):
  index_names=data[data['SOG']==0.0].index
  data.drop(index_names, inplace=True)

In [20]:
def vesseltype_categories(data):
  data.dropna(subset = ["VesselType"], axis = 0, inplace=True)
  data['VesselType'] = data['VesselType'].apply(np.int64)
  data["VesselType"].replace({0: "Not Available", 30: "Fishing", 35:"Military", 
                          (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,24,25,26,27,28,29,33,34, 
                           38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59, 90,
                           91,92,93,94,95,96,97,98,99):'Other',
                          (60,61,62,63,64,65,66,67,68,69):'Passenger',
                          (70,71,72,73,74,75,76,77,78,79):'Cargo',
                          (36,37):'Pleasure Craft/Sailing',
                          (80,81,82,83,84,85,86,87,88,89):'Tanker',
                          (21,22,31,32,52):'Tug Tow'}, inplace=True)
  label_encoder = LabelEncoder()
  data['VesselType_labeled']=label_encoder.fit_transform(data['VesselType'])

In [21]:
def course_direction(data):
  def categorize(data):
    if data['COG']<22.5:
       return 'N'
    if data['COG']>=22.5 and data['COG']<67.5:
      return 'NE'
    if data['COG']>=67.5 and data['COG']<112.5:
      return 'E'
    if data['COG']>=112.5 and data['COG']<157.5:
      return 'SE'
    if data['COG']>=157.5 and data['COG']<202.5:
      return 'S'
    if data['COG']>=202.5 and data['COG']<247.5:
      return 'SW'
    if data['COG']>=247.5 and data['COG']<292.5:
      return 'W'
    if data['COG']>=292.5 and data['COG']<337.5:
      return 'NW'
    if data['COG']>=337.5:
      return 'N'
  data['COG_direction'] = data.apply(lambda data: categorize(data), axis=1)

In [22]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371000):
  if to_radians:
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
  a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
  return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [23]:
def apply_haversine(data):
  data= data.sort_values(['MMSI', 'BaseDateTime'], ascending = True)
  data['distance'] = np.concatenate(data.groupby('MMSI').apply(lambda x: haversine(x['LAT'], x['LON'], x['LAT'].shift(), x['LON'].shift())).values)
  data['distance'] = data['distance'].fillna(0)
  data['dist_diff'] = data['distance'].diff()
  indexNames = data[ data['dist_diff'] <= 500 ].index # Get names of indexes for which column dist_diff has value less than or equal to 1
  data.drop(indexNames , inplace=True) # Delete these row indexes from dataFrame
  data['dist_diff'] = data['dist_diff'].fillna(0)

In [24]:
def geohash(data):
  data['geohash']=data.apply(lambda x: gh.encode(x.LAT, x.LON, precision=5), axis=1)

In [25]:
def Eda(data):
  drop_missing_values(data)
  drop_unwanted_columns(data)
  drop_invalid_mmsi(data)
  zero_sog(data)
  vesseltype_categories(data)
  geohash(data)
  course_direction(data)
  apply_haversine(data)
  print(data)


def readFiles():  
  length = len(my_list)
  for i in range(length):
    Eda(my_list[i])
readFiles()

              MMSI         BaseDateTime       LAT        LON   SOG    COG  \
3        316005971  2022-01-01T00:00:06  46.50268  -84.35674   2.4  258.6   
4        316004054  2022-01-01T00:00:07  46.50326  -84.37506   0.3   61.9   
48       368038000  2022-01-01T00:00:08  48.52225 -122.61212   0.1  169.2   
53       271046486  2022-01-01T00:00:04  37.67832  -74.47989  11.3  206.9   
54       367712860  2022-01-01T00:00:08  47.53809 -122.32879   0.1  232.5   
...            ...                  ...       ...        ...   ...    ...   
7239746  372188000  2022-01-01T21:44:44  13.46064  144.66639   0.1  226.1   
7239749  372188000  2022-01-01T22:05:44  13.46061  144.66644   0.1  181.1   
7239750  372188000  2022-01-01T22:23:44  13.46064  144.66643   0.1  193.2   
7239754  372188000  2022-01-01T23:23:44  13.46062  144.66645   0.1   57.6   
7239755  372188000  2022-01-01T23:29:44  13.46063  144.66643   0.1   47.7   

        VesselType  VesselType_labeled geohash COG_direction  
3          T

In [27]:
import os
dir_name = '/content/drive/MyDrive/zip/csv1'

extension = ".csv"

folderpath = "/content/drive/MyDrive/zip/csv1/backup"
#os.mkdir(folderpath)
for i, df in enumerate(my_list, 1):
    filename = "AIS_2022_01_{}.csv".format(i)
    filepath = os.path.join(folderpath, filename)
    df.to_csv(filepath)
    #os.chdir('/content/drive/MyDrive/zip/csv1') # change directory from working dir to dir with files

    #for item in os.listdir(dir_name): 
     # if item.endswith(extension): 
         # file_name = os.path.abspath(item)
   # os.remove(file_name)

In [28]:
new_file_list = glob.glob('/content/drive/MyDrive/zip/csv1/backup/*.csv')
print(new_file_list)

['/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_1.csv', '/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_2.csv', '/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_3.csv', '/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_4.csv', '/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_5.csv', '/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_6.csv']


In [29]:
df_list = (pd.read_csv(file) for file in new_file_list)

In [30]:
big_df = pd.concat(map(pd.read_csv, glob.glob( '/content/drive/MyDrive/zip/csv1/backup/*.csv')))

In [31]:
big_df.shape

(5569004, 11)

In [32]:
grouped_big_df = big_df.groupby(['geohash', 'VesselType', 'SOG']).agg({'VesselType':'count'})
grouped_big_df.columns = ['count']
grouped_big_df = grouped_big_df.reset_index()
grouped_big_df.head(100)

,geohash,VesselType,SOG,count
0,860p8,Cargo,11.1,1
1,87nxs,Cargo,20.6,1
2,87nxs,Cargo,21.0,1
3,87nxw,Cargo,20.8,1
4,87nxx,Cargo,21.0,1
...,...,...,...,...
95,87pzv,Cargo,22.7,1
96,87pzv,Cargo,22.8,1
97,87pzv,Cargo,22.9,1
98,87pzy,Cargo,22.5,2


In [33]:
def my_agg(x):
  names = {'weighted_average_SOG': (x['count']*x['SOG']).sum()/x['count'].sum()}
  return pd.Series(names, index=['weighted_average_SOG'])

In [34]:
grouped_big_df.groupby(['geohash', 'VesselType']).apply(my_agg)

,,weighted_average_SOG
geohash,VesselType,
860p8,Cargo,11.100000
87nxs,Cargo,20.800000
87nxw,Cargo,20.800000
87nxx,Cargo,21.000000
87nz9,Cargo,21.433333
...,...,...
x604c,Tanker,11.733333
x604d,Tanker,11.800000
x604h,Tanker,11.800000


In [40]:
df = pd.read_csv('/content/drive/MyDrive/zip/csv1/backup/AIS_2022_01_6.csv')
df.head()

,Unnamed: 0,MMSI,BaseDateTime,LAT,LON,SOG,COG,VesselType,VesselType_labeled,geohash,COG_direction
0,0,215165000,2022-01-05T00:00:00,30.47683,-118.10385,2.1,126.0,Cargo,0,9mkj2,SE
1,9,311349000,2022-01-05T00:00:05,48.11061,-122.49679,11.2,100.2,Cargo,0,c28cs,E
2,19,367362010,2022-01-05T00:00:06,38.02591,-122.38158,11.9,238.5,Tug Tow,8,9qbbr,SW
3,27,636020110,2022-01-05T00:00:01,25.43902,-79.85176,14.2,23.4,Cargo,0,dhx0w,NE
4,50,368027230,2022-01-05T00:00:00,47.95763,-122.29866,12.9,146.5,Passenger,5,c290c,SE
